## EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_delay = pd.read_csv('/Users/cheyen/Desktop/projet_certification/BLOC_5/dataset/get_around_delay_analysis.csv')
data_pricing = pd.read_csv('/Users/cheyen/Desktop/projet_certification/BLOC_5/dataset/get_around_pricing_project.csv')

In [3]:
data_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Unnamed: 7,Unnamed: 8
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN,NaN,NaN


In [4]:
df_delay = data_delay.copy()

In [5]:
# Deleting Unnamed columns
df_delay.drop(['Unnamed: 7', 'Unnamed: 8'], axis=1, inplace=True)

In [6]:
df_delay.shape

(21310, 7)

In [7]:
df_delay.describe(include='all')

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


delay_at_checkout_in_minutes show us the difference in minutes between the rental end time that the driver has chosen at the time of booking and the real time the driver completed the checkout. In this column we can see that the "max" value is 71084 minutes late which is almost 50 days... Same for the "min" value : - 22433 means that the driver return the car 15 days in advance.

Lot of missing values in this column too, it could be interesting to see if the NaN values represent the 0 value which means that the driver has returned the car on time. 

For missing values in previous_ended_rental_id it only means that there was no previous rental or the delay was higher than 12 hours. 

For missing values in time_delta_with_previous_rental_in_minutes, it means that the difference in minutes between two rentals was higher than 12 hours.

Some outliers and necessary to check missing values in delay_at_checkout_in_minutes

In [8]:
# Checking missing values
df_delay.isna().sum()

rental_id                                         0
car_id                                            0
checkin_type                                      0
state                                             0
delay_at_checkout_in_minutes                   4964
previous_ended_rental_id                      19469
time_delta_with_previous_rental_in_minutes    19469
dtype: int64

In [9]:
df_zero = df_delay[df_delay['delay_at_checkout_in_minutes'] == 0]
df_zero

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
314,550660,399967,mobile,ended,0.0,NaN,NaN
576,531451,378003,mobile,ended,0.0,NaN,NaN
583,532695,320824,connect,ended,0.0,NaN,NaN
810,529269,361156,mobile,ended,0.0,NaN,NaN
1337,540574,408117,mobile,ended,0.0,NaN,NaN
...,...,...,...,...,...,...,...
20402,566064,267977,mobile,ended,0.0,NaN,NaN
21006,559537,209458,mobile,ended,0.0,NaN,NaN
21008,559676,232442,mobile,ended,0.0,NaN,NaN
21186,567477,362507,connect,ended,0.0,NaN,NaN


It seems that NaN values in the "delay..." column are real missing values 

In [10]:
print(df_delay['state'].value_counts())
print()
print(df_delay['checkin_type'].value_counts())

state
ended       18045
canceled     3265
Name: count, dtype: int64

checkin_type
mobile     17003
connect     4307
Name: count, dtype: int64


In [11]:
delay_checkout = []
for delay in df_delay['delay_at_checkout_in_minutes']:
    if delay < 0:
        delay_checkout.append('Early')
    elif delay < 15:
        delay_checkout.append('Late 0-15')
    elif delay < 30:
        delay_checkout.append('Late 15-30')
    elif delay < 60:
        delay_checkout.append('Late 30-60')
    elif delay < 120:
        delay_checkout.append('Late 60-120')
    elif delay >= 120:
        delay_checkout.append('Very late')
    else:
        delay_checkout.append('NA')
    
df_delay['delay'] = delay_checkout
df_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NA
1,507750,269550,mobile,ended,-81.0,NaN,NaN,Early
2,508131,359049,connect,ended,70.0,NaN,NaN,Late 60-120
3,508865,299063,connect,canceled,NaN,NaN,NaN,NA
4,511440,313932,mobile,ended,NaN,NaN,NaN,NA


To remove outliers I decide to delete all rentals checked out more than one day late and rentals checked out more than one day early. 

In [12]:
outliers = df_delay[(df_delay['delay_at_checkout_in_minutes'] > 1440) | (df_delay['delay_at_checkout_in_minutes'] < -1440)]
outliers

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay
24,521191,224745,mobile,ended,1452.0,NaN,NaN,Very late
102,537082,359641,mobile,ended,1967.0,NaN,NaN,Very late
114,538012,192803,mobile,ended,3233.0,NaN,NaN,Very late
153,540733,393558,mobile,ended,15422.0,NaN,NaN,Very late
574,530971,241568,mobile,ended,3371.0,NaN,NaN,Very late
...,...,...,...,...,...,...,...,...
20947,554377,318954,mobile,ended,-1468.0,NaN,NaN,Early
21002,559126,379544,mobile,canceled,-17468.0,NaN,NaN,Early
21101,565287,415234,mobile,ended,2824.0,NaN,NaN,Very late
21104,565590,397046,mobile,ended,1728.0,NaN,NaN,Very late


In [13]:
df_delay = df_delay.drop(outliers.index)
df_delay.shape

(21081, 8)

In [16]:
df_delay['next_rental'] = [True if x > 0 else False for x in df_delay['previous_ended_rental_id']]
df_delay.drop(['previous_ended_rental_id'], axis=1, inplace=True)
df_delay

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,time_delta_with_previous_rental_in_minutes,delay,next_rental
0,505000,363965,mobile,canceled,NaN,NaN,NA,False
1,507750,269550,mobile,ended,-81.0,NaN,Early,False
2,508131,359049,connect,ended,70.0,NaN,Late 60-120,False
3,508865,299063,connect,canceled,NaN,NaN,NA,False
4,511440,313932,mobile,ended,NaN,NaN,NA,False
...,...,...,...,...,...,...,...,...
21305,573446,380069,mobile,ended,NaN,300.0,NA,True
21306,573790,341965,mobile,ended,-337.0,NaN,Early,False
21307,573791,364890,mobile,ended,144.0,NaN,Very late,False
21308,574852,362531,connect,ended,-76.0,NaN,Early,False
